In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType, ComplexType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, eval_pointvalues, project
from utils.plotting import Mpl2DPlotter, Mpl2DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "./results/2d-homogeneous-with-dissipation"
animator_dir = f'{save_dir}/animator'
Path(save_dir).mkdir(parents=True, exist_ok=True)
Path(animator_dir).mkdir(parents=True, exist_ok=True)

U_PATH = f"{save_dir}/uj.xdmf"
UD_PATH = f"{save_dir}/ujd.xdmf"
PRESSURE_PATH = f"{save_dir}/pressure.xdmf"

In [3]:
def generate_boundaries(points):
    return [(1, lambda x: np.isclose(x[0], points[0][0])),
            (2, lambda x: np.isclose(x[0], points[0][1])),
            (3, lambda x: np.isclose(x[1], points[1][0])),
            (4, lambda x: np.isclose(x[1], points[1][1])),]

In [4]:
def problem_setup(N: int,
                  points,
                  T: float,
                  dt: float,
                  fluid: str):
    """
    Performs problem configuration w.r.t. given parameters
    """
    
    # Mesh and function space definition
    domain = mesh.create_rectangle(
        MPI.COMM_WORLD, 
        [(points[0][0], points[1][0]), (points[0][1], points[1][1])], 
        N
    )
    V = fem.VectorFunctionSpace(domain, ("CG", 2))

    u = TrialFunction(V)
    v = TestFunction(V)
    
    # Definition functions of physical characteristics
    ro, c, eta = fluids.loc[fluid, ['Density', 'Speed of sound', 'Viscosity']]
        
    # Construction of problem form
    GAMMA, BETA = 0.5, 0.5
    
    mm = ScalarType(ro) * inner(u, v) * dx
    aa = ScalarType(ro * c**2) * inner(grad(u), grad(v)) * dx
    cc = ScalarType(4./3 * eta) * inner(grad(u), grad(v)) * dx
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt**2 * BETA * aa)
    
    # Definition of boundary conditions
    boundaries = generate_boundaries(points)
    measure = generate_boundary_measure(boundaries, domain)
    
    u_D = lambda x: [x[0] * 0.0, x[1] * 0.0]
    u_N1 = fem.Constant(domain, ScalarType((-1, 0)))
    u_N2 = fem.Constant(domain, ScalarType((0, 0)))

    bcs = [BoundaryCondition("Dirichlet", 1, u_D, V, u, v, measure).bc,
           BoundaryCondition("Dirichlet", 3, u_D, V, u, v, measure).bc, 
           BoundaryCondition("Dirichlet", 4, u_D, V, u, v, measure).bc]
    
    nbcs = [BoundaryCondition("Neumann", 2, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 2, u_N2, V, u, v, measure).bc] 
    
    return {
        'Params': (N, points, fluid, ro, c, T, dt, GAMMA, BETA),
        'FunctionSpace': (domain, V, u, v),
        'Forms': (mm, aa, cc), 
        'Problem': (F, bcs, nbcs)
    }

In [5]:
def solve_problem(config):
    N, points, fluid, ro, c, T, dt, GAMMA, BETA = config['Params']
    domain, V, u, v = config['FunctionSpace']
    mm, aa, cc = config['Forms']
    F, bcs, nbcs = config['Problem']
    
    xdmf_u = io.XDMFFile(domain.comm, U_PATH, "w")
    xdmf_ud = io.XDMFFile(domain.comm, UD_PATH, "w")
    xdmf_p = io.XDMFFile(domain.comm, PRESSURE_PATH, "w")
    xdmf_u.write_mesh(domain)
    xdmf_ud.write_mesh(domain)
    xdmf_p.write_mesh(domain)
    
    # Create initial condition
    initial_condition = lambda x: [x[0] * 0.0, x[1] * 0.0]
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_dtg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_dtg.interpolate(initial_condition)
    
    p = project(ro * c**2 * div(uj), domain, ("CG", 1))
    
    # Construct the right hand side of the problem
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(PETSc.KSP.Type.PREONLY)
    solver.getPC().setType(PETSc.PC.Type.LU)

    # Solve problem at each time step
    num_steps = int(T / dt)
    t = 0.0
    
    time_data = {
        't'   : [0.0,],
        'uj'  : [np.copy(uj.x.array[:]),],
        'uj_d': [np.copy(uj_d.x.array[:]),],
        'p'   : [np.copy(p.x.array[:]),]
    }
    
    for i in range(num_steps):
        t += dt

        # Multiply values of uj_d by dt and GAMMA, 
        # because form does not support muliplication by constant
        uj_d_dtg.x.array[:] = uj_d.x.array * dt * GAMMA

        # Construct the right hand side of the problem at each time step
        idx = 0 if t < 1.0e-6 else 1

        L = nbcs[idx] + (cc * uj_d) + (aa * uj) + (aa * uj_d_dtg)
        linear_form = fem.form(rhs(L))

        b = fem.petsc.create_vector(linear_form)
        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # Apply Dirichlet boundary condition to the vector
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, bcs)

        # Solve linear problem
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()

        # Update solution
        uj1_d_array = uj_d.x.array + dt * uj_dd.x.array

        uj.x.array[:] = uj.x.array + 0.5 * dt * (uj_d.x.array + uj1_d_array)
        uj_d.x.array[:] = uj1_d_array

        # Save results into the files
        if i % 5 == 0:
            p = project(ro * c**2 * div(uj), domain, ("CG", 1))

            xdmf_u.write_function(uj, t)
            xdmf_ud.write_function(uj_d, t)
            xdmf_p.write_function(p, t)
            
            if i % 25 == 0:
                time_data['t'].append(t)
                time_data['uj'].append(np.copy(uj.x.array))
                time_data['uj_d'].append(np.copy(uj_d.x.array))
                time_data['p'].append(np.copy(p.x.array))
        
    xdmf_u.close()
    xdmf_ud.close()
    xdmf_p.close()
    
    return uj, uj_d, p, time_data

#### Available fluids

In [6]:
fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
fluids

,Density,Speed of sound,Viscosity,Heat,Thermal conductivity,Thermal expansion
Fluid,,,,,,
Water,1000,1500,0.000894,4182.0,0.598,0.00015
Fuel oil,890,1360,2.022000,NaN,NaN,NaN
Oil,760,1470,0.000500,NaN,NaN,NaN
Glycerin,1260,1905,1.500000,NaN,NaN,NaN


In [7]:
config = problem_setup(N=[20, 20], 
                       points=[[0.0, 0.01], [0.0, 0.01]],
                       T=8.0e-6,
                       dt=5.0e-9,
                       fluid='Glycerin')

In [8]:
uj, uj_d, p, time_data = solve_problem(config)

In [9]:
mpa = Mpl2DAnimator(layout=[['1', '2'], ['3', '3']], time=time_data['t'], projection='2d')
mpa.update_figure(figsize=(16, 12), fontsize=20)
mpa.add_fun(fun=uj, time_data=time_data['uj'], type='real', project='mag', 
            points=config['Params'][1], N=[20, 20], axes_id='1', title='Displacement')
mpa.add_fun(fun=uj_d, time_data=time_data['uj_d'], type='real', project='mag', 
            points=config['Params'][1], N=[20, 20], axes_id='2', title='Acceleration')
mpa.add_fun(fun=p, time_data=time_data['p'], type='real', project='scalar', 
            points=config['Params'][1], N=[20, 20], axes_id='3', title='Pressure')
mpa.write(path=animator_dir, filename='animation_2d.mp4', fps=10, interval=2000)

In [10]:
Video(f'{animator_dir}/animation_2d.mp4', width=800)

In [11]:
mpa = Mpl2DAnimator(layout=[['1', '2'], ['3', '']], time=time_data['t'], projection='3d')
mpa.update_figure(figsize=(16, 12), fontsize=20)
mpa.add_fun(fun=uj, time_data=time_data['uj'], type='real', project='mag', 
            points=config['Params'][1], N=[20, 20], axes_id='1', title='Displacement')
mpa.add_fun(fun=uj_d, time_data=time_data['uj_d'], type='real', project='mag', 
            points=config['Params'][1], N=[20, 20], axes_id='2', title='Acceleration')
mpa.add_fun(fun=p, time_data=time_data['p'], type='real', project='scalar', 
            points=config['Params'][1], N=[20, 20], axes_id='3', title='Pressure')
mpa.write(path=animator_dir, filename='animation_3d.mp4', fps=10, interval=2000)

In [12]:
Video(f'{animator_dir}/animation_3d.mp4', width=800)